<a href="https://colab.research.google.com/github/head4ths/NLP/blob/master/NLP_PJ_615E_GROUP5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW 확인


## GPU 확인

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('GPU 할당 필요')
else:
  print(gpu_info)

Mon Jun 29 04:11:37 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 메모리 확인

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('RAM 사이즈 {:.1f} GB'.format(ram_gb))

RAM 사이즈 27.4 GB


## 기타 사양 확인 

In [3]:
!cat /etc/issue.net

Ubuntu 18.04.3 LTS


In [4]:
!head /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2300.000
cache size	: 46080 KB
physical id	: 0


In [5]:
!head -n 3 /proc/meminfo

MemTotal:       26751700 kB
MemFree:        23888956 kB
MemAvailable:   25726684 kB


In [6]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay          69G   32G   34G  49% /
tmpfs            64M     0   64M   0% /dev
tmpfs            13G     0   13G   0% /sys/fs/cgroup
shm              13G     0   13G   0% /dev/shm
/dev/sda1        75G   33G   43G  44% /opt/bin
tmpfs            13G   16K   13G   1% /var/colab
tmpfs            13G     0   13G   0% /proc/acpi
tmpfs            13G     0   13G   0% /proc/scsi
tmpfs            13G     0   13G   0% /sys/firmware


In [7]:
!python --version

Python 3.6.9


# 환경설정

## 튜닝용 파라미터

In [8]:
gv_epoch = 3
gv_MAX_LEN = 64
gv_batch_size = 10  
gv_model_nm = 'bert-large-cased'
gv_do_lower_case = False
gv_lr = 4e-6
gv_train_data = "NLP/friends_train.json"

gv_num_warmup_steps = 0.2
gv_eps = 1e-8
gv_seed_val = 2019517005
gv_random_state = 20200616

## 라이브러리 설치

In [9]:
!pip install transformers

     |████████████████████████████████| 675kB 2.8MB/s 
     |████████████████████████████████| 1.1MB 14.5MB/s 
     |████████████████████████████████| 890kB 20.9MB/s 
     |████████████████████████████████| 3.8MB 23.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=6b58f24a14b6037d39d93626a355618093689ba4415ce7515c2824698ea5da76
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


## 라이브러리 import

In [10]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

from google.colab import files
from imblearn.over_sampling import RandomOverSampler

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


# 데이터셋 준비

## raw data 준비 (github 사용시)

In [11]:
!git clone https://github.com/head4ths/NLP.git

Cloning into 'NLP'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 21 (delta 8), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (21/21), done.


In [12]:
!ls NLP -la

total 6728
drwxr-xr-x 3 root root    4096 Jun 29 04:12 .
drwxr-xr-x 1 root root    4096 Jun 29 04:12 ..
-rw-r--r-- 1 root root  201615 Jun 29 04:12 en_data.csv
-rw-r--r-- 1 root root   45048 Jun 29 04:12 en_sample.csv
-rw-r--r-- 1 root root 2839207 Jun 29 04:12 friends_all.json
-rw-r--r-- 1 root root  229392 Jun 29 04:12 friends_dev.json
-rw-r--r-- 1 root root  544341 Jun 29 04:12 friends_test.json
-rw-r--r-- 1 root root 2065470 Jun 29 04:12 friends_train.json
drwxr-xr-x 8 root root    4096 Jun 29 04:12 .git
-rw-r--r-- 1 root root  846591 Jun 29 04:12 ko_data.csv
-rw-r--r-- 1 root root   89587 Jun 29 04:12 ko_sample.csv


## JSON to pandas load

In [13]:
pd_train = pd.read_json(gv_train_data, encoding="utf-8") 
pd_test = pd.read_json("NLP/friends_test.json", encoding="utf-8") 
pd_submit = pd.read_csv("NLP/en_data.csv") 

print(pd_train.shape)
print(pd_test.shape)
print(pd_submit.shape)
pd_submit["emotion"] = "neutral"

print(pd_train[0][0])
print(pd_test[0][0])
print(pd_submit.head())




(720, 24)
(200, 24)
(3296, 5)
{'speaker': 'Chandler', 'utterance': 'also I was the point person on my company\x92s transition from the KL-5 to GR-6 system.', 'emotion': 'neutral', 'annotation': '4100000'}
{'speaker': 'Mark', 'utterance': 'Why do all you\x92re coffee mugs have numbers on the bottom?', 'emotion': 'surprise', 'annotation': '2000030'}
   id  i_dialog  ...                                          utterance  emotion
0   0         0  ...                      Alright, whadyou do with him?  neutral
1   1         0  ...                                  Oh! You're awake!  neutral
2   2         0  ...  Then you gotta come clean with Ma! This is not...  neutral
3   3         0  ...                                  Yeah, but this is  neutral
4   4         0  ...          I don't wanna hear it! Now go to my room!  neutral

[5 rows x 6 columns]


In [14]:
pd_submit

,id,i_dialog,i_utterance,speaker,utterance,emotion
0,0,0,0,Phoebe,"Alright, whadyou do with him?",neutral
1,1,0,1,Monica,Oh! You're awake!,neutral
2,2,0,2,Joey,Then you gotta come clean with Ma! This is not...,neutral
3,3,0,3,Mr. Tribbiani,"Yeah, but this is",neutral
4,4,0,4,Joey,I don't wanna hear it! Now go to my room!,neutral
...,...,...,...,...,...,...
3291,3291,239,18,Monica,I guess.,neutral
3292,3292,239,19,Chandler,"So, shouldn?t we go give her the benefit of th...",neutral
3293,3293,239,20,Monica,Fine. I?m just glad I didn?t give her my secre...,neutral
3294,3294,239,21,Chandler,"Out of curiosity, what is your secret ingredient?",neutral


## Flatten


In [15]:
def convert_input_data_flat(p_data):

  x = len(p_data.columns)
  y = len(p_data[0])

  print("convert_input_data_flat(x,y) : " + str(x) + "," +str(y) )  

  pd_xy = pd.DataFrame()

  id = 0 
  i_utterance = 0

  for j in range(y):
    i_utterance = 0
    pd_x = pd.DataFrame()
    for i in range(x):
      #print(i)

      if p_data.loc[j,i] != None:
        pd_x = pd_x.append(pd.DataFrame.from_dict([p_data.loc[j,i]])  )
        id += 1
        i_utterance += 1

    pd_x.insert(0, "i_dialog", j, True)
    pd_x.insert(1, "i_utterance", range(0,i_utterance), True)
    pd_x.drop(['annotation'],axis=1,inplace=True)      
    pd_xy = pd_xy.append(pd_x)

  pd_xy.insert(0, "id", range(0,id), True)    
  pd_xy.index = pd_xy["id"]

  return pd_xy


In [16]:
train = convert_input_data_flat(pd_train)
train["utterance"] = train["speaker"] + " : " + train["utterance"]
train.drop(['speaker'],axis=1,inplace=True)

test = convert_input_data_flat(pd_test)
test["utterance"] = test["speaker"] + " : " + test["utterance"]
test.drop(['speaker'],axis=1,inplace=True)

submit = pd.DataFrame(pd_submit)
submit["utterance"] = submit["speaker"] + " : " + submit["utterance"]
submit.drop(['speaker'],axis=1,inplace=True)

print(train.shape)
print(test.shape)
print(submit.shape)

convert_input_data_flat(x,y) : 24,720
convert_input_data_flat(x,y) : 24,200
(10561, 5)
(2764, 5)
(3296, 5)


In [17]:
grouped = train.groupby(train['emotion'])
grouped.count()

,id,i_dialog,i_utterance,utterance
emotion,,,,
anger,513,513,513,513
disgust,240,240,240,240
fear,185,185,185,185
joy,1283,1283,1283,1283
neutral,4752,4752,4752,4752
non-neutral,2017,2017,2017,2017
sadness,351,351,351,351
surprise,1220,1220,1220,1220


In [18]:
num_labels = {
    "anger": 0,
    "disgust": 1,
    "fear": 2,
    "joy": 3,
    "neutral": 4,
    "non-neutral": 5,
    "sadness": 6,
    "surprise": 7
}

labels_num = {
    0 : "anger",
    1 : "disgust",
    2 : "fear",
    3 : "joy",
    4 : "neutral",
    5 : "non-neutral",
    6 : "sadness",
    7 : "surprise"
}


In [19]:
train['emotion'] = train['emotion'].replace(num_labels)
test['emotion'] = test['emotion'].replace(num_labels)
submit['emotion'] = submit['emotion'].replace(num_labels)

In [20]:
train.head()

,id,i_dialog,i_utterance,utterance,emotion
id,,,,,
0,0,0,0,Chandler : also I was the point person on my c...,4
1,1,0,1,The Interviewer : You mustve had your hands f...,4
2,2,0,2,Chandler : That I did. That I did.,4
3,3,0,3,The Interviewer : So lets talk a little bit a...,4
4,4,0,4,Chandler : My duties? All right.,7


In [21]:
test.head()

,id,i_dialog,i_utterance,utterance,emotion
id,,,,,
0,0,0,0,Mark : Why do all youre coffee mugs have numb...,7
1,1,0,1,Rachel : Oh. Thats so Monica can keep track. ...,5
2,2,0,2,Rachel : Y'know what?,4
3,3,0,3,Ross : It didnt.,4
4,4,0,4,"Frank : Okay, so what you used to have with Ra...",3


In [22]:
submit.head()

,id,i_dialog,i_utterance,utterance,emotion
0,0,0,0,"Phoebe : Alright, whadyou do with him?",4
1,1,0,1,Monica : Oh! You're awake!,4
2,2,0,2,Joey : Then you gotta come clean with Ma! This...,4
3,3,0,3,"Mr. Tribbiani : Yeah, but this is",4
4,4,0,4,Joey : I don't wanna hear it! Now go to my room!,4


## Oversampleing

In [23]:
"""
ros = RandomOverSampler(random_state=42) 

print(train.shape)

train_tmp = train.copy()
oversampled_data, oversampled_label = ros.fit_resample(train_tmp,train_tmp["emotion"])
train = pd.DataFrame(oversampled_data, columns=train_tmp.columns) 
print(train.shape)
"""

'\nros = RandomOverSampler(random_state=42) \n\nprint(train.shape)\n\ntrain_tmp = train.copy()\noversampled_data, oversampled_label = ros.fit_resample(train_tmp,train_tmp["emotion"])\ntrain = pd.DataFrame(oversampled_data, columns=train_tmp.columns) \nprint(train.shape)\n'

In [24]:
train.head()

,id,i_dialog,i_utterance,utterance,emotion
id,,,,,
0,0,0,0,Chandler : also I was the point person on my c...,4
1,1,0,1,The Interviewer : You mustve had your hands f...,4
2,2,0,2,Chandler : That I did. That I did.,4
3,3,0,3,The Interviewer : So lets talk a little bit a...,4
4,4,0,4,Chandler : My duties? All right.,7


In [25]:
train['emotion'] = train['emotion'].astype('int64')
train['emotion']

id
0        4
1        4
2        4
3        4
4        7
        ..
10556    4
10557    5
10558    7
10559    4
10560    5
Name: emotion, Length: 10561, dtype: int64

In [26]:
grouped = train.groupby(train['emotion'])
grouped.count()

,id,i_dialog,i_utterance,utterance,emotion
emotion,,,,,
0,513,513,513,513,513
1,240,240,240,240,240
2,185,185,185,185,185
3,1283,1283,1283,1283,1283
4,4752,4752,4752,4752,4752
5,2017,2017,2017,2017,2017
6,351,351,351,351,351
7,1220,1220,1220,1220,1220


In [27]:
print(type(train['emotion']))
print(type(test['emotion']))
print(type(submit['emotion']))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


# 전처리 - Traning Set 

## 형식 변환 [CLS] [SEP]

In [28]:
sentences = train['utterance']
sentences[:10]

id
0    Chandler : also I was the point person on my c...
1    The Interviewer : You mustve had your hands f...
2                   Chandler : That I did. That I did.
3    The Interviewer : So lets talk a little bit a...
4                    Chandler : My duties?  All right.
5    The Interviewer : Now youll be heading a whol...
6                                    Chandler : I see.
7    The Interviewer : But therell be perhaps 30 p...
8                             Chandler : Good to know.
9              The Interviewer : We can go into detail
Name: utterance, dtype: object

In [29]:
sentences_temp = sentences.copy()

for i in range(len(sentences)):
  if (i == 0 ): 
    sentences_temp[i] = "[CLS] " + str(sentences[i]) + " [SEP] [None] [SEP]" 
  elif (train['i_dialog'][i-1] !=  train['i_dialog'][i]):  
    sentences_temp[i] = "[CLS] " + str(sentences[i]) + " [SEP] [None] [SEP]" 
  else:    
    sentences_temp[i] = "[CLS] " + str(sentences[i]) + " [SEP] " + str(sentences[i-1]) + " [SEP]"

sentences = sentences_temp

In [30]:
sentences[:10]

id
0    [CLS] Chandler : also I was the point person o...
1    [CLS] The Interviewer : You mustve had your h...
2    [CLS] Chandler : That I did. That I did. [SEP]...
3    [CLS] The Interviewer : So lets talk a little...
4    [CLS] Chandler : My duties?  All right. [SEP] ...
5    [CLS] The Interviewer : Now youll be heading ...
6    [CLS] Chandler : I see. [SEP] The Interviewer ...
7    [CLS] The Interviewer : But therell be perhap...
8    [CLS] Chandler : Good to know. [SEP] The Inter...
9    [CLS] The Interviewer : We can go into detail ...
Name: utterance, dtype: object

In [31]:
labels = train["emotion"].values
labels

array([4, 4, 4, ..., 7, 4, 5])

## BERT 토크나이징

In [32]:
tokenizer = BertTokenizer.from_pretrained(gv_model_nm, do_lower_case=gv_do_lower_case)  
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print (sentences[0])
print (tokenized_texts[0])


[CLS] Chandler : also I was the point person on my companys transition from the KL-5 to GR-6 system. [SEP] [None] [SEP]
['[CLS]', 'Chandler', ':', 'also', 'I', 'was', 'the', 'point', 'person', 'on', 'my', 'company', '##s', 'transition', 'from', 'the', 'K', '##L', '-', '5', 'to', 'G', '##R', '-', '6', 'system', '.', '[SEP]', '[', 'None', ']', '[SEP]']


In [33]:
MAX_LEN = gv_MAX_LEN
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
input_ids[0]

array([  101, 14394,   131,  1145,   146,  1108,  1103,  1553,  1825,
        1113,  1139,  1419,  1116,  6468,  1121,  1103,   148,  2162,
         118,   126,  1106,   144,  2069,   118,   127,  1449,   119,
         102,   164,  7330,   166,   102,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0])

## 어텐션 마스크

In [34]:
attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [35]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=gv_random_state, 
                                                                                    test_size=0.1)

In [36]:
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=gv_random_state, 
                                                       test_size=0.1)

In [37]:
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels) 
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels) 
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([ 101, 9300,  131, 1398, 1268,  117,  146,  112, 1325, 1267, 1128, 1107,
        1103, 2106,  119,  102, 8958,  131,  146, 1221,  119,  102,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0])
tensor(4)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([  101, 19704,   131,  7066,   118,  5871,   106,  6466, 11012,  1183,
          106,   102,  9300,   131,  1573,   146,  1169,  1202,  1122,   136,
          102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0

## 배치 사이즈

In [38]:
batch_size = gv_batch_size

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

# 전처리 - Test Set 

## 형식 변환 [CLS] [SEP]

In [39]:
sentences = test['utterance']
sentences[:10]

id
0    Mark : Why do all youre coffee mugs have numb...
1    Rachel : Oh. Thats so Monica can keep track. ...
2                                Rachel : Y'know what?
3                                    Ross : It didnt.
4    Frank : Okay, so what you used to have with Ra...
5              Joey : Now, wh-what, what is that like?
6    Frank : Its so cool man, its so, its just ...
7                                   Ross : Yeah, yeah.
8                        Joey : Why cant I find that?
9         Ross : Dont ask me, I had it and I blew it!
Name: utterance, dtype: object

In [40]:
sentences_temp = sentences.copy()

for i in range(len(sentences)):
  if (i == 0 ): 
    sentences_temp[i] = "[CLS] " + str(sentences[i]) + " [SEP] [None] [SEP]" 
  elif (test['i_dialog'][i-1] !=  test['i_dialog'][i]):  
    sentences_temp[i] = "[CLS] " + str(sentences[i]) + " [SEP] [None] [SEP]" 
  else:    
    sentences_temp[i] = "[CLS] " + str(sentences[i]) + " [SEP] " + str(sentences[i-1]) + " [SEP]"

sentences = sentences_temp

In [41]:
sentences[:10]

id
0    [CLS] Mark : Why do all youre coffee mugs hav...
1    [CLS] Rachel : Oh. Thats so Monica can keep t...
2    [CLS] Rachel : Y'know what? [SEP] Rachel : Oh....
3    [CLS] Ross : It didnt. [SEP] Rachel : Y'know ...
4    [CLS] Frank : Okay, so what you used to have w...
5    [CLS] Joey : Now, wh-what, what is that like? ...
6    [CLS] Frank : Its so cool man, its so, its ...
7    [CLS] Ross : Yeah, yeah. [SEP] Frank : Its so...
8    [CLS] Joey : Why cant I find that? [SEP] Ross...
9    [CLS] Ross : Dont ask me, I had it and I blew...
Name: utterance, dtype: object

In [42]:
labels = test["emotion"].values
labels 

array([7, 5, 4, ..., 4, 4, 4])

## BERT 토크나이징

In [43]:
tokenizer = BertTokenizer.from_pretrained(gv_model_nm, do_lower_case=gv_do_lower_case)  
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] Mark : Why do all youre coffee mugs have numbers on the bottom? [SEP] [None] [SEP]
['[CLS]', 'Mark', ':', 'Why', 'do', 'all', 'your', '##e', 'coffee', 'mug', '##s', 'have', 'numbers', 'on', 'the', 'bottom', '?', '[SEP]', '[', 'None', ']', '[SEP]']


In [44]:
MAX_LEN = gv_MAX_LEN
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
input_ids[0]

array([  101,  2392,   131,  2009,  1202,  1155,  1240,  1162,  3538,
       15761,  1116,  1138,  2849,  1113,  1103,  3248,   136,   102,
         164,  7330,   166,   102,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0])

## 어텐션 마스크

In [45]:
attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [46]:
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([  101,  2392,   131,  2009,  1202,  1155,  1240,  1162,  3538, 15761,
         1116,  1138,  2849,  1113,  1103,  3248,   136,   102,   164,  7330,
          166,   102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])
tensor(7)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


## 배치 사이즈

In [47]:
batch_size = gv_batch_size
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

# 전처리 - Submit Set

## 형식 변환 [CLS][SEP]

In [48]:
submit

,id,i_dialog,i_utterance,utterance,emotion
0,0,0,0,"Phoebe : Alright, whadyou do with him?",4
1,1,0,1,Monica : Oh! You're awake!,4
2,2,0,2,Joey : Then you gotta come clean with Ma! This...,4
3,3,0,3,"Mr. Tribbiani : Yeah, but this is",4
4,4,0,4,Joey : I don't wanna hear it! Now go to my room!,4
...,...,...,...,...,...
3291,3291,239,18,Monica : I guess.,4
3292,3292,239,19,"Chandler : So, shouldn?t we go give her the be...",4
3293,3293,239,20,Monica : Fine. I?m just glad I didn?t give her...,4
3294,3294,239,21,"Chandler : Out of curiosity, what is your secr...",4


In [49]:
sentences = submit['utterance']
sentences[:10]

0               Phoebe : Alright, whadyou do with him?
1                           Monica : Oh! You're awake!
2    Joey : Then you gotta come clean with Ma! This...
3                    Mr. Tribbiani : Yeah, but this is
4     Joey : I don't wanna hear it! Now go to my room!
5    Chandler : I don?t want him to tell this story...
6    Ross : Oh, but he will. He still tells the sto...
7                          Monica : I wasn?t escaping.
8    Ross : Then how did you get caught in the barb...
9        Monica : I was trying to help out a squirrel.
Name: utterance, dtype: object

In [50]:
sentences_temp = sentences.copy()

for i in range(len(sentences)):
  if (i == 0 ): 
    sentences_temp[i] = "[CLS] " + str(sentences[i]) + " [SEP] [None] [SEP]" 
  elif (submit['i_dialog'][i-1] !=  submit['i_dialog'][i]):  
    sentences_temp[i] = "[CLS] " + str(sentences[i]) + " [SEP] [None] [SEP]" 
  else:    
    sentences_temp[i] = "[CLS] " + str(sentences[i]) + " [SEP] " + str(sentences[i-1]) + " [SEP]"

sentences = sentences_temp

In [51]:
sentences[:10]

0    [CLS] Phoebe : Alright, whadyou do with him? [...
1    [CLS] Monica : Oh! You're awake! [SEP] Phoebe ...
2    [CLS] Joey : Then you gotta come clean with Ma...
3    [CLS] Mr. Tribbiani : Yeah, but this is [SEP] ...
4    [CLS] Joey : I don't wanna hear it! Now go to ...
5    [CLS] Chandler : I don?t want him to tell this...
6    [CLS] Ross : Oh, but he will. He still tells t...
7    [CLS] Monica : I wasn?t escaping. [SEP] Ross :...
8    [CLS] Ross : Then how did you get caught in th...
9    [CLS] Monica : I was trying to help out a squi...
Name: utterance, dtype: object

In [52]:
labels = submit["emotion"].values
labels 

array([4, 4, 4, ..., 4, 4, 4])

## BERT 토크나이징


In [53]:
tokenizer = BertTokenizer.from_pretrained(gv_model_nm, do_lower_case=gv_do_lower_case)  
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] Phoebe : Alright, whadyou do with him? [SEP] [None] [SEP]
['[CLS]', 'Phoebe', ':', 'Alright', ',', 'w', '##hady', '##ou', 'do', 'with', 'him', '?', '[SEP]', '[', 'None', ']', '[SEP]']


In [54]:
MAX_LEN = gv_MAX_LEN
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
input_ids[0]

array([  101, 19704,   131, 18009,   117,   192, 24905,  6094,  1202,
        1114,  1140,   136,   102,   164,  7330,   166,   102,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0])

## 어텐션 마스크

In [55]:
attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [56]:
submit_inputs = torch.tensor(input_ids)
submit_labels = torch.tensor(labels)
submit_masks = torch.tensor(attention_masks)

print(submit_inputs[0])
print(submit_labels[0])
print(submit_masks[0])

tensor([  101, 19704,   131, 18009,   117,   192, 24905,  6094,  1202,  1114,
         1140,   136,   102,   164,  7330,   166,   102,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])
tensor(4)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


## 배치 사이즈

In [57]:
batch_size = gv_batch_size
submit_data = TensorDataset(submit_inputs, submit_masks, submit_labels)
submit_sampler = SequentialSampler(submit_data)
submit_dataloader = DataLoader(submit_data, sampler=submit_sampler, batch_size=batch_size, shuffle=False)

# 모델 생성

## 사전 설정

In [58]:
device_name = tf.test.gpu_device_name()

if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [59]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


## 모델 생성

In [60]:
model = BertForSequenceClassification.from_pretrained(gv_model_nm, num_labels=8)
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1

## 옵티마이저 및 에폭 설정

In [61]:
optimizer = AdamW(model.parameters(), lr = gv_lr, eps = gv_eps )
epochs = gv_epoch
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = gv_num_warmup_steps,
                                            num_training_steps = total_steps)

# 모델 학습 - Traning Set

## 함수 정의

In [62]:
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [63]:
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

## 학습 수행

In [64]:
seed_val = gv_seed_val
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
model.zero_grad()

In [65]:
t0 = time.time()
total_loss = 0
model.train()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1

In [66]:
for step, batch in enumerate(train_dataloader):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
    
    outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)

In [67]:
for epoch_i in range(0, epochs):
    t0 = time.time()
    total_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        model.zero_grad()

    avg_train_loss = total_loss / len(train_dataloader)           
    t0 = time.time()
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():     
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
            
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  예상 Score : {0:.3f}".format(eval_accuracy/nb_eval_steps))


  예상 Score : 0.632
  예상 Score : 0.664
  예상 Score : 0.666


# 평가 - Submit Set

In [68]:
t0 = time.time()
model.eval()
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

submit_preds =  []

for step, batch in enumerate(submit_dataloader):

    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():     
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
        
    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

    if step < 10:
      print(step)
      print(submit_preds[0:10])
      print(logits.argmax(-1))    
    submit_preds.extend(logits.argmax(-1))     
    if step < 10:
      print(submit_preds[0:10])   
    
print("예상 Score : {0:.3f}".format(eval_accuracy/nb_eval_steps))

0
[]
[4 7 0 4 0 5 4 5 4 4]
[4, 7, 0, 4, 0, 5, 4, 5, 4, 4]
1
[4, 7, 0, 4, 0, 5, 4, 5, 4, 4]
[5 4 4 4 3 7 6 7 4 4]
[4, 7, 0, 4, 0, 5, 4, 5, 4, 4]
2
[4, 7, 0, 4, 0, 5, 4, 5, 4, 4]
[4 4 7 5 3 0 0 0 0 0]
[4, 7, 0, 4, 0, 5, 4, 5, 4, 4]
3
[4, 7, 0, 4, 0, 5, 4, 5, 4, 4]
[0 4 6 4 4 5 4 4 7 4]
[4, 7, 0, 4, 0, 5, 4, 5, 4, 4]
4
[4, 7, 0, 4, 0, 5, 4, 5, 4, 4]
[4 4 7 7 4 7 6 7 4 4]
[4, 7, 0, 4, 0, 5, 4, 5, 4, 4]
5
[4, 7, 0, 4, 0, 5, 4, 5, 4, 4]
[4 4 4 4 4 4 7 3 5 4]
[4, 7, 0, 4, 0, 5, 4, 5, 4, 4]
6
[4, 7, 0, 4, 0, 5, 4, 5, 4, 4]
[7 4 5 4 4 5 4 7 0 3]
[4, 7, 0, 4, 0, 5, 4, 5, 4, 4]
7
[4, 7, 0, 4, 0, 5, 4, 5, 4, 4]
[7 4 4 3 4 0 0 0 3 0]
[4, 7, 0, 4, 0, 5, 4, 5, 4, 4]
8
[4, 7, 0, 4, 0, 5, 4, 5, 4, 4]
[4 7 4 3 4 3 3 4 4 4]
[4, 7, 0, 4, 0, 5, 4, 5, 4, 4]
9
[4, 7, 0, 4, 0, 5, 4, 5, 4, 4]
[4 4 4 4 0 5 4 4 4 4]
[4, 7, 0, 4, 0, 5, 4, 5, 4, 4]
예상 Score : 0.516


In [69]:
print(np.argmax(logits))

36


In [70]:
submit_preds[0:10]

[4, 7, 0, 4, 0, 5, 4, 5, 4, 4]

In [71]:
logits.argmax(-1)

array([5, 4, 4, 5, 4, 3])

In [72]:
print([labels_num[submit_pred] for submit_pred in submit_preds])

['neutral', 'surprise', 'anger', 'neutral', 'anger', 'non-neutral', 'neutral', 'non-neutral', 'neutral', 'neutral', 'non-neutral', 'neutral', 'neutral', 'neutral', 'joy', 'surprise', 'sadness', 'surprise', 'neutral', 'neutral', 'neutral', 'neutral', 'surprise', 'non-neutral', 'joy', 'anger', 'anger', 'anger', 'anger', 'anger', 'anger', 'neutral', 'sadness', 'neutral', 'neutral', 'non-neutral', 'neutral', 'neutral', 'surprise', 'neutral', 'neutral', 'neutral', 'surprise', 'surprise', 'neutral', 'surprise', 'sadness', 'surprise', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'surprise', 'joy', 'non-neutral', 'neutral', 'surprise', 'neutral', 'non-neutral', 'neutral', 'neutral', 'non-neutral', 'neutral', 'surprise', 'anger', 'joy', 'surprise', 'neutral', 'neutral', 'joy', 'neutral', 'anger', 'anger', 'anger', 'joy', 'anger', 'neutral', 'surprise', 'neutral', 'joy', 'neutral', 'joy', 'joy', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'ne

# 평가 - Test Set 

In [73]:
t0 = time.time()
model.eval()
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

for step, batch in enumerate(test_dataloader):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():     
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
        
    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("예상 Score : {0:.3f}".format(eval_accuracy/nb_eval_steps))

예상 Score : 0.627


# 캐글용 csv 생성

In [74]:
submit_pred_labels = [labels_num[submit_pred] for submit_pred in submit_preds]

In [75]:
submission_dic = {"id":list(range(len(submit_pred_labels))), "Predicted":submit_pred_labels} 
submission_df = pd.DataFrame(submission_dic) 
submission_df.to_csv("DFC615E 20061X X.csv", index=False)
files.download("DFC615E 20061X X.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>